## Simple benchmark using XGB

In [1]:
import os
import sys
import pandas as pd

sys.path.insert(0, os.path.abspath('/home/jupyter/kaggle/predict_future_sales/src/'))

import common.com_util as util
import config.constants as constants
import cv.cv_util as cv
import munging.process_data as process_data
import modeling.train_util as model

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
logger = util.get_logger('sub_2')

train_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/train_all_merged.feather')
test_df = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/test_all_merged.feather')

train_features = ['shop_id', 'item_id', 'date_block_num', 'item_category_id', 'item_cnt_month']
train_df = train_df[train_features]

test_features = ['ID', 'shop_id', 'item_id', 'date_block_num', 'item_category_id']
test_df = test_df[test_features]

sample_submission = pd.read_feather('/home/jupyter/kaggle/predict_future_sales/data/processed/submission_processed.feather')

test = test_df.drop(['ID'], axis='columns')

# Clip the item_cnt_month within 20
train_df.item_cnt_month.clip(lower=0, upper=20, inplace=True)

/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [4]:
TARGET = 'item_cnt_month'
ID = 'ID'
SEED = 42

training_months = [0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
holdout_months = [33]

training, validation = cv.get_data_splits_by_date_block(logger, train_df, 
                                                        train_months=training_months, 
                                                        validation_months=holdout_months)

[INFO]2020-09-04 06:50:00,331:sub_2:Splitting the data into train and holdout based on months...
[INFO]2020-09-04 06:50:00,331:sub_2:Training months [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32]
[INFO]2020-09-04 06:50:00,333:sub_2:Validation months [33]
[INFO]2020-09-04 06:50:01,097:sub_2:Shape of the training data (10675678, 5) 
[INFO]2020-09-04 06:50:01,098:sub_2:Shape of the validation data (238172, 5)


In [5]:
xgb_params = {
                # Learning task parameters
                'objective': 'reg:squarederror',
                'eval_metric': 'rmse',
                'seed': SEED,

                # Type of the booster
                'booster': 'gbtree',
                
                # parameters for tree booster
                'learning_rate': 0.3,
                'max_depth': 6,
                'max_leaves': 0,
                'max_bin': 256,
                'early_stopping_rounds' : 100,

                # General parameters
                'verbosity': 2, #info
                }

In [6]:
predictors = ['shop_id', 'item_id', 'date_block_num', 'item_category_id']

In [ ]:
bst, validation_score = model.xgb_train_validate_on_holdout(
    logger=logger, training=training, validation=validation, 
    predictors=predictors, target=TARGET, params=xgb_params, test_X=None, 
    n_estimators=10000, early_stopping_rounds=100, verbose_eval=100
)

[INFO]2020-09-04 06:50:01,201:sub_2:Training using XGBoost and validating on holdout
[INFO]2020-09-04 06:50:01,276:sub_2:Shape of train_X : (10675678, 4)
[INFO]2020-09-04 06:50:01,277:sub_2:Shape of train_Y : (10675678,)
[INFO]2020-09-04 06:50:01,278:sub_2:Shape of validation_X : (238172, 4)
[INFO]2020-09-04 06:50:01,279:sub_2:Shape of validation_Y : (238172,)


/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/jupyter/anaconda3/envs/py37/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


[06:50:05] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/learner.cc:686: Tree method is automatically selected to be 'approx' for faster speed. To use old behavior (exact greedy algorithm on single machine), set tree_method to 'exact'.
[06:50:18] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124 extra nodes, 0 pruned nodes, max_depth=6
[0]	train-rmse:1.1954	valid_data-rmse:1.12987
Multiple eval metrics have been passed: 'valid_data-rmse' will be used for early stopping.

Will train until valid_data-rmse hasn't improved in 100 rounds.
[06:50:28] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 126 extra nodes, 0 pruned nodes, max_depth=6
[06:50:39] INFO: /home/conda/feedstock_root/build_artifacts/xgboost_1566327313563/work/src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 124

In [ ]:
bst.best_iteration, bst.best_score

In [51]:
bst.best_score['valid_0']['rmse']

1.0364347687294806

In [43]:
prediction = bst.predict(test, bst.best_iteration)

In [44]:
len(prediction)

214200

In [45]:
test_df.shape

(214200, 5)

In [46]:
sample_submission.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [28]:
submission = pd.DataFrame({'ID': test_df.ID, 'item_cnt_month': prediction})

In [36]:
submission.to_csv('')

ID                0
item_cnt_month    0
dtype: int64